In [60]:
import pandas as pd
import numpy as np  
import os 
import boto3 
import gzip
from dotenv import load_dotenv, find_dotenv

In [61]:
dotenv_path = find_dotenv()
load_dotenv(dotenv_path)
s3_secret_key = os.environ['AWS_SECRET_ACCESS_KEY']
s3_bucket_name = os.environ['JUPYTERHUB_USER']
s3_endpoint_url = os.environ['S3_ENDPOINT_URL']
s3_access_key = os.environ['AWS_ACCESS_KEY_ID']
s3bucket = os.environ['BUCKET']

print(s3_endpoint_url)
print(s3_bucket_name)
s3 = boto3.client('s3','us-east-1', endpoint_url= s3_endpoint_url,
                       aws_access_key_id = s3_access_key,
                       aws_secret_access_key = s3_secret_key)

https://s3.upshift.redhat.com
cdolfi


In [80]:
#CHANGE WITH WHAT YOU ARE UPDATING 
folder = "user"
files = []

for key in s3.list_objects(Bucket=s3_bucket_name)['Contents']:
    if key['Key'][:4] == folder:
        files.append(key['Key'])
        
obj = s3.get_object(Bucket=s3_bucket_name, Key = files[0])  
combined_emails = pd.read_csv(obj['Body'])
        
for f in files[1:]:
    temp = s3.get_object(Bucket=s3_bucket_name, Key = f) 
    n_df = pd.read_csv(temp['Body'])
    combined_emails = pd.concat([combined_emails, n_df])
    
    
combined_emails.sort_values(by= 'datetime', inplace = True)
combined_emails.drop('Unnamed: 0',axis=1, inplace=True )


In [82]:
#update complete csv 
combined_emails.to_csv('../data/interim/temp_complete.csv')
s3_location = folder + "/" + folder + "_complete.csv"
s3.upload_file(Filename='../data/interim/temp_complete.csv',Bucket=s3_bucket_name, Key=s3_location)

In [78]:
#remove merged files
for k in files:
    if k != s3_location:
        s3.delete_object(Bucket=s3_bucket_name, Key=k)

To check what items are in the bucket

In [79]:
for key in s3.list_objects(Bucket=s3_bucket_name)['Contents']:
    print(key['Key'])

devl/devl_complete.csv
user/2004_3-2005_1.csv
user/2005_2-2006_1.csv
user/2006_02-2006_05.csv
user/2006_06-2006_07.csv
user/2006_07-2006_08.csv
user/2006_09-2007_01.csv
user/2007_01-2007_06.csv
user/2007_07-2008_01.csv
user/2008_01-2009_01.csv
user/2009_01-2009_06.csv
user/2009_07-2010_01.csv
user/2010_01-2010_01.csv
user/2010_01-2011_01.csv
user/2011_01-2012_01.csv
user/2012_01-2013_01.csv
user/2013_01-2014_01.csv
user/2014_01-2015_01.csv
user/2016_01-2017_01.csv
user/2017_01-2018_01.csv
user/2018_01-2019_01.csv
user/2019_01-2020_01.csv
user/2020_01-2021_01.csv
user/user_complete.csv
